# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Kerbal model written by Bjorn Cole

The textual SysML v2 model is the Kerbal model:

    package Kerbal {
        package 'Rocket Building' {
            import ScalarFunctions::*;

            part def Rocket {
                part stages : 'Rocket Stage' [1..5] {
                    // placing this here because previous stages only make sense in context of a full vehicle
                    ref 'Carried Stage' : 'Rocket Stage' [1];
                    attribute 'Payload Mass' : Real;
                    attribute 'Loaded Mass' : Real;
                    attribute 'Burnout Mass' : Real;
                    part 'Coupler to Carrying Stage' : Coupler [0..8] {
                        attribute 'Separation Force' : Real;
                    }
                }
            }
            item def Oxidizer;
            item def Fuel;
            item def 'Solid Propellant';

            // TODO: Something something part symmetry
            abstract part def 'Rocket Stage';

            // use stage types to enforce matching
            part def 'Liquid Stage' :> 'Rocket Stage' {
                part engines : 'Liquid Engine' [0..8];
                part tanks : 'Fuel Tank Section' [0..30];
                attribute 'Full Mass' : Real = sum(engines->collect p:'Kerbal Rocket Part' (p::Mass)) +
                                        sum(tanks->collect p:'Fuel Tank Section' (p::'Full Mass'));

                attribute 'Empty Mass' : Real;
            }
            part def 'Solid Stage' :> 'Rocket Stage' {
                part boosters : 'Solid Booster' [0..8];
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            part def 'Coupler' :> 'Kerbal Rocket Part';

            abstract part def 'Fuel Tank Section' :> 'Kerbal Rocket Part' {
                attribute 'Liquid Fuel' : Real;
                attribute 'Oxidizer' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
            }

            abstract part def 'Liquid Engine' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Thrust' : Real;
            }
            abstract part def 'Solid Booster' :> 'Kerbal Rocket Part' {
                attribute 'Specific Impulse' : Real;
                attribute 'Full Mass' : Real;
                attribute 'Empty Mass' : Real;
                attribute 'Thrust' : Real;
            }

            part def 'Pod' :> 'Kerbal Rocket Part'  {
                attribute Torque : Real;
            }
            part def 'Parachute' :> 'Kerbal Rocket Part';

            part def 'Kerbal Rocket Part' {
                attribute Mass : Real;
                attribute 'Max Temperature' : Real;
            }
        }
        package 'Parts Library' {
            import ScalarFunctions::*;
            part def 'FL-T200 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 1.125;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.125;
            }
            part def 'FL-T100 Fuel Tank' :> 'Rocket Building'::'Fuel Tank Section' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Full Mass' = 0.5625;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Fuel Tank Section'::'Empty Mass' = 0.0625;
            }
            part def 'Mk1 Command Pod' :> 'Rocket Building'::'Pod';
            part def 'LV-T45 "Swivel" Liquid Fuel Engine' :> 'Rocket Building'::'Liquid Engine' {
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Liquid Engine'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Liquid Engine'::'Thrust' = 167.97;
                attribute 'Mass' : Real :>> 'Rocket Building'::'Kerbal Rocket Part'::Mass = 1.50;
            }
            part def 'RT-5 "Flea" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 1.50;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.45;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 140.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::'Thrust' = 162.91;
            }
            part def 'RT-10 "Hammer" Solid Fuel Booster' :> 'Rocket Building'::'Solid Booster' {
                attribute 'Full Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Full Mass' = 3.56;
                attribute 'Empty Mass' : Real :>> 'Rocket Building'::'Solid Booster'::'Empty Mass' = 0.75;
                attribute 'Specific Impulse' : Real :>> 'Rocket Building'::'Solid Booster'::'Specific Impulse' = 170.0;
                attribute 'Thrust' : Real :>> 'Rocket Building'::'Solid Booster'::Thrust = 197.90;
            }
        }
    }

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as nx
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.client import SysML2Client
from pymbe.query.metamodel_navigator import map_inputs_to_results

from pymbe.interpretation.calc_dependencies import generate_execution_order

import os

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [2]:
fts_full_mass = '004a1b5f-4bfc-4460-9f38-1e7b4caba6e5' # Full Mass Attribute under Fuel Tank Sections
ft200_full_mass = '1e5a0ed7-8b41-4ab4-a433-8f7eedd75833' # Full Mass Attribute under FL-T200 Fuel Tank
ft100_full_mass = 'a57b423b-5c0c-4057-be6b-689abcb536b2' # Full Mass Attribute under FL-T100 Fuel Tank
liquid_stage_full_mass = '7beafac8-c1c1-4b1b-ae21-d3c9a733531c' # Full Mass Attribute under Liquid Stage
top_plus = 'b51bb349-e210-4be8-be64-e749ea4e563b' # The '+' Expression under the Full Mass Attribute under Liquid Stage
tank_mass_sum_1 = '700d97d1-410a-459c-ad09-8792c27e2803' # The 'sum' Expression that sums Full Mass of tanks
collect_1 = 'd6644a0a-6eef-49c1-a770-60886073554c' # The 'collect' Expression that gathers Full Masses that apply to the current scope
collect_1_result = '2caccce7-a0b4-4926-8f24-0dbffb92f6ad' # The result parameter of above collect expression
full_mass_dot = 'ad0bff53-eebe-4446-a8df-4db0b7187707' # Expression that scopes Full Mass attribute in FeatureReferenceExpression
fre_1 = '2665fb1b-1f12-4f13-a977-0f060915773e' # Expression to point to the instances of Full Mass
fre_1_result = '6cfb516b-6045-454e-a521-83b747acef7e' # Result of gathering all instances of Full Mass from the interpretation
sum_1_result = '31f8c4bd-9700-4bc3-9970-3eb5451f0203' # Result of the sum Expression on Full Mass
x = '478a8a4d-90c7-44fd-a2ae-e4057636d4bc'

fre_2_result = '5897d247-1e81-41ad-bc5e-92eac8b35c2f' # Result of gathering all instances of Full Mass from the interpretation
collect_2_result = '2119c27e-d44c-479e-8b31-64de015dad61' # The result parameter of above collect expression
sum_2_collection = '3947bb94-0f09-413a-924b-2b1422354c93' # Input to sum Expression for Mass
sum_2_result = 'e50e47ab-4d6c-42d1-8997-81c52ffb7068' # Result of the sum Expression on Mass

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [3]:
helper_client = SysML2Client()

path = "C:\\Users\\bjorn\\Documents\\Git\\pyMBE"
file_name = "\\tests\\data\\Kerbal\\elements.json"

helper_client._load_disk_elements(path + str(file_name))

Create a graph representation of the model and load it into memory.

In [4]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [5]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [6]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake
)

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [7]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

['FL-T100 Fuel Tank', 'Full Mass: Real'], 151
['FL-T100 Fuel Tank', 'Empty Mass: Real'], 151
['Rocket', 'stages: Rocket Stage', 'Payload Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Coupler to Carrying Stage: Coupler', 'Separation Force: Real'], 40
['Rocket', 'stages: Rocket Stage', 'Loaded Mass: Real'], 5
['Rocket', 'stages: Rocket Stage', 'Burnout Mass: Real'], 5
['Pod', 'Torque: Real'], 1
['LV-T45 "Swivel" Liquid Fuel Engine', 'Mass: Real'], 41
['LV-T45 "Swivel" Liquid Fuel Engine', 'Specific Impulse: Real'], 41
['LV-T45 "Swivel" Liquid Fuel Engine', 'Thrust: Real'], 41
['RT-5 "Flea" Solid Fuel Booster', 'Thrust: Real'], 41
['RT-5 "Flea" Solid Fuel Booster', 'Full Mass: Real'], 41
['RT-5 "Flea" Solid Fuel Booster', 'Specific Impulse: Real'], 41
['RT-5 "Flea" Solid Fuel Booster', 'Empty Mass: Real'], 41
['Liquid Stage', 'Full Mass: Real'], 5
['Liquid Stage', 'Empty Mass: Real'], 5
['Liquid Stage', 'engines: Liquid Engine'], 40
['Liquid Stage', 'tanks: Fuel Tank Section'], 150


Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [8]:
dcg = generate_execution_order(lpg, m0_interpretation)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'ImpliedParameterFeedforward'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [9]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [10]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

(8 «Occurred LiteralInteger», LS#0.8 «Occurred LiteralInteger».$result#0 (unset), Output)
(162.91 «Occurred LiteralReal», RT-5#0.162.91 «Occurred LiteralReal».$result#0 (unset), Output)
(197.9 «Occurred LiteralReal», RT-10#0.197.9 «Occurred LiteralReal».$result#0 (unset), Output)
(RT-5#0.162.91 «Occurred LiteralReal».$result#0 (unset), .Real#1512 (unset), ValueBinding)
(RT-10#0.197.9 «Occurred LiteralReal».$result#0 (unset), .Real#394 (unset), ValueBinding)
(.Real#1512 (unset), .Real#1356 (unset), Redefinition)
(.Real#394 (unset), .Real#1356 (unset), Redefinition)
(0.5625 «Occurred LiteralReal», FL-T100#0.0.5625 «Occurred LiteralReal».$result#0 (unset), Output)
(1.125 «Occurred LiteralReal», FL-T200#0.1.125 «Occurred LiteralReal».$result#0 (unset), Output)
(1.5 «Occurred LiteralReal», LV-T45#0.1.5 «Occurred LiteralReal».$result#0 (unset), Output)
(FL-T100#0.0.5625 «Occurred LiteralReal».$result#0 (unset), .Real#2570 (unset), ValueBinding)
(FL-T200#0.1.125 «Occurred LiteralReal».$result

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [11]:
cg.solve_graph(lpg)

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [12]:
m0_interpretation[collect_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#0 ([.Real#1713 (1.125), .Real#964 (1.125), .Real#1345 (1.125), .Real#938 (0.5625), .Real#1551 (0.5625), .Real#2505 (0.5625), .Real#1235 (1.125), .Real#2369 (1.125), .Real#599 (0.5625), .Real#2 (0.5625), .Real#594 (0.5625), .Real#1962 (1.125), .Real#2451 (1.125), .Real#1900 (1.125), .Real#2381 (0.5625), .Real#1923 (0.5625), .Real#515 (1.125), .Real#2603 (1.125), .Real#1991 (1.125), .Real#720 (1.125), .Real#1146 (1.125)])]]

In [13]:
m0_interpretation[collect_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#0 ([.Real#2650 (1.5), .Real#617 (1.5), .Real#1697 (1.5)])]]

In [14]:
m0_interpretation[sum_2_collection]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$collection#0 ([.Real#2650 (1.5), .Real#617 (1.5), .Real#1697 (1.5)])]]

In [15]:
m0_interpretation[sum_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$result#0 (19.125)]]

In [16]:
m0_interpretation[x]

[[LS#0, + ($x, $y) => $result, LS#0.+ ($x, $y) => $result.$x#0 (4.5)]]

In [17]:
m0_interpretation[liquid_stage_full_mass]

[[LS#0, .Real#885 (23.625)]]

In [18]:
m0_interpretation[fre_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Full Mass (p) => $result,
  FRE.Full Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Full Mass (p) => $result.FRE.Full Mass.$result#0 ([[FL-T100#0, .Real#1341 (0.5625)], [FL-T100#1, .Real#118 (0.5625)], [FL-T100#2, .Real#1696 (0.5625)], [FL-T100#3, .Real#754 (0.5625)], [FL-T100#4, .Real#1923 (0.5625)], [FL-T100#5, .Real#1895 (0.5625)], [FL-T100#6, .Real#392 (0.5625)], [FL-T100#7, .Real#2505 (0.5625)], [FL-T100#8, .Real#2599 (0.5625)], [FL-T100#9, .Real#1524 (0.5625)], [FL-T100#10, .Real#1963 (0.5625)], [FL-T100#11, .Real#334 (0.5625)], [FL-T100#12, .Real#29 (0.5625)], [FL-T100#13, .Real#2368 (0.5625)], [FL-T100#14, .Real#2055 (0.5625)], [FL-T100#15, .Real#737 (0.5625)], [FL-T100#16, .Real#1307 (0.5625)], [FL-T100#17, .Real#124 (0.5625)], [FL-T100#18, .Real#1874 (0.5625)], [FL-T100#19, .Real#705 (0.5625)], [FL-T100#20, .Real#260 

In [19]:
m0_interpretation[fre_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#0 ([[Coupler#0, .Real#829 (unset)], [Coupler#1, .Real#2678 (unset)], [Coupler#2, .Real#1722 (unset)], [Coupler#3, .Real#839 (unset)], [Coupler#4, .Real#1350 (unset)], [Coupler#5, .Real#2325 (unset)], [Coupler#6, .Real#445 (unset)], [Coupler#7, .Real#330 (unset)], [Coupler#8, .Real#121 (unset)], [Coupler#9, .Real#81 (unset)], [Coupler#10, .Real#2164 (unset)], [Coupler#11, .Real#2468 (unset)], [Coupler#12, .Real#2062 (unset)], [Coupler#13, .Real#1108 (unset)], [Coupler#14, .Real#632 (unset)], [Coupler#15, .Real#971 (unset)], [Coupler#16, .Real#1148 (unset)], [Coupler#17, .Real#129 (unset)], [Coupler#18, .Real#1627 (unset)], [Coupler#19, .Real#1834 (unset)], [Coupler#20, .Real#1798 (unset)], [Coupler#21, .Real#1130 (unset

Show all interpretation sequence sets (limited to length of 5).

In [20]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)

Real, id = ede2b2e7-9280-4932-9453-134bf460892f, size = 2695
[.Real#0 (0.0625)]
[.Real#1 (unset)]
[.Real#2 (0.5625)]
[.Real#3 (unset)]
[.Real#4 (unset)]
['..']
Coupler, id = 9e6e4ffc-1ad9-4351-bfb6-882de5c73c74, size = 40
[Coupler#0]
[Coupler#1]
[Coupler#2]
[Coupler#3]
[Coupler#4]
['..']
FL-T100 Fuel Tank, id = 1eca9960-e445-4d2f-be3f-cd7a6882435d, size = 60
[FL-T100#0]
[FL-T100#1]
[FL-T100#2]
[FL-T100#3]
[FL-T100#4]
['..']
FL-T200 Fuel Tank, id = cc585eec-c66c-48aa-b319-1395a0c8e292, size = 90
[FL-T200#0]
[FL-T200#1]
[FL-T200#2]
[FL-T200#3]
[FL-T200#4]
['..']
RT-5 "Flea" Solid Fuel Booster, id = 5be56a39-f4a4-4fbb-872c-12f3e717593c, size = 27
[RT-5#0]
[RT-5#1]
[RT-5#2]
[RT-5#3]
[RT-5#4]
['..']
RT-10 "Hammer" Solid Fuel Booster, id = 8851ab1c-0d7f-4fe2-bee0-8b29d408c897, size = 13
[RT-10#0]
[RT-10#1]
[RT-10#2]
[RT-10#3]
[RT-10#4]
['..']
LV-T45 "Swivel" Liquid Fuel Engine, id = 21aa0007-73df-4f80-a292-678b6e3bd735, size = 40
[LV-T45#0]
[LV-T45#1]
[LV-T45#2]
[LV-T45#3]
[LV-T45#4]
['..']
